In [34]:
import xarray as xr
import numpy as np

### Test mit MSLP

#### import and rename

In [35]:
mslp_test = xr.open_mfdataset('/mnt/c/Users/susaw/Desktop/JRA-55_subsets/mslp/*.nc',
                       chunks={'initial_time0_hours':504,'g0_lat_1':29,'g0_lon_2':29})
mslp_test = mslp_test.rename({'initial_time0_hours':'time','g0_lat_1':'lat','g0_lon_2':'lon',
                'PRMSL_GDS0_MSL':'mslp'})
mslp_test.coords['lon'] = (mslp_test.coords['lon'] + 180) % 360 - 180
mslp_test=mslp_test.drop("initial_time0_encoded")
#print(mslp_test)

#### Tageswerte

In [36]:
daily_mslp = mslp_test.resample(time='1D').mean().chunk({'time':504,'lat':29,'lon':29})
print(daily_mslp)

<xarray.Dataset>
Dimensions:  (lat: 29, lon: 29, time: 1461)
Coordinates:
  * time     (time) datetime64[ns] 1958-01-01 1958-01-02 ... 1961-12-31
  * lat      (lat) float32 67.5 66.25 65.0 63.75 62.5 ... 36.25 35.0 33.75 32.5
  * lon      (lon) float32 -10.0 -8.75 -7.5 -6.25 -5.0 ... 21.25 22.5 23.75 25.0
Data variables:
    mslp     (time, lat, lon) float32 dask.array<shape=(1461, 29, 29), chunksize=(504, 29, 29)>


#### gleitendes Mittel über 21 Tage + Klimatologien

In [37]:
dailyroll_mslp = daily_mslp.rolling(time=21,center=True).mean()
climatology=dailyroll_mslp.groupby("time.dayofyear").mean('time')
print(climatology)

<xarray.Dataset>
Dimensions:    (dayofyear: 366, lat: 29, lon: 29)
Coordinates:
  * lat        (lat) float32 67.5 66.25 65.0 63.75 ... 36.25 35.0 33.75 32.5
  * lon        (lon) float32 -10.0 -8.75 -7.5 -6.25 ... 21.25 22.5 23.75 25.0
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 360 361 362 363 364 365 366
Data variables:
    mslp       (dayofyear, lat, lon) float32 dask.array<shape=(366, 29, 29), chunksize=(1, 29, 29)>


### Anomalien

In [38]:
anomalie_mslp = dailyroll_mslp.groupby('time.dayofyear') - climatology

#### Normieren der Anomalien

In [39]:
std_mslp = climatology.groupby('dayofyear').std()
nomano_mslp = anomalie_mslp.groupby('time.dayofyear')/std_mslp

/home/susa/anaconda3/envs/env1/lib/python3.7/site-packages/xarray/core/groupby.py:758: FutureWarning: Default reduction dimension will be changed to the grouped dimension in a future version of xarray. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  allow_lazy=True, **kwargs)


In [40]:
print (nomano_mslp)

<xarray.Dataset>
Dimensions:    (lat: 29, lon: 29, time: 1461)
Coordinates:
  * lat        (lat) float32 67.5 66.25 65.0 63.75 ... 36.25 35.0 33.75 32.5
  * lon        (lon) float32 -10.0 -8.75 -7.5 -6.25 ... 21.25 22.5 23.75 25.0
  * time       (time) datetime64[ns] 1958-01-01 1958-01-02 ... 1961-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
Data variables:
    mslp       (time, lat, lon) float32 dask.array<shape=(1461, 29, 29), chunksize=(1, 29, 29)>
